# Defining Tools

A `Tool` consists of:
1. A name that is used to uniquely identify the tool so the LM can specify which tool it intends to call
2. A description that is used to help indicate to the LM when it is appropriate to call the tool
3. A callable that is executed when the tool is actually called
4. A constraint that ensures that the LM's *output* is a valid *input* to the callable

## Functions
The most common type of tool is defined by a function.

The constraint on the LM's output can be specified by a JSON schema, where the JSON schema should have type `object` and have `properties` matching the names of the function's arguments.

Note: positional-only and variadic arguments are not yet supported.

In [1]:
import random
from typing import Literal

def get_weather_func(city: str, unit: Literal["celsius", "fahrenheit"] = "celsius") -> str:
    """
    Get the current weather for a given city.
    """
    temp = random.randint(-10, 35)
    if temp <= 0:
        weather = "snowy"
    elif temp <= 20:
        weather = "cloudy"
    else:
        weather = "sunny"
    if unit == "fahrenheit":
        temp = temp * 9/5 + 32
    return f"The current temperature in {city} is {temp} degrees {unit} and it is {weather}."

`Tool.from_callable` takes a function and creates a `Tool` object.

- The name of the tool will be inferred from the function's name
- The description of the tool will be inferred from the function's docstring
- The JSON schema of the parameters will be inferred from any type annotations

In [2]:
from guidance.types import Tool

Tool.from_callable(
    callable=get_weather_func
).model_dump()

{'name': 'get_weather_func',
 'description': 'Get the current weather for a given city.',
 'tool': {'type': 'function',
  'parameters': {'additionalProperties': False,
   'properties': {'city': {'title': 'City', 'type': 'string'},
    'unit': {'enum': ['celsius', 'fahrenheit'],
     'title': 'Unit',
     'type': 'string'}},
   'required': ['city', 'unit'],
   'title': 'get_weather_func',
   'type': 'object'}},
 'callable': <function __main__.get_weather_func(city: str, unit: Literal['celsius', 'fahrenheit'] = 'celsius') -> str>}

The name, description, and parameters (may be a JSON schema or a pydantic.BaseModel class) can optionally be specified directly: 

In [3]:
from pydantic import BaseModel, Field, ConfigDict

class GetWeatherArgs(BaseModel):
    city: str = Field(..., description="The name of the city to get the weather for.")
    unit: Literal["celsius", "fahrenheit"] = Field("celsius", description="The unit of temperature to return.")

    model_config = ConfigDict(
        extra="forbid", # Disallow extra fields -- required for OpenAI
    )

get_weather_tool = Tool.from_callable(
    callable=get_weather_func,
    name="get_weather",
    description="Get the current weather for a given city.",
    parameters=GetWeatherArgs
)
get_weather_tool.model_dump()

{'name': 'get_weather',
 'description': 'Get the current weather for a given city.',
 'tool': {'type': 'function',
  'parameters': {'additionalProperties': False,
   'properties': {'city': {'description': 'The name of the city to get the weather for.',
     'title': 'City',
     'type': 'string'},
    'unit': {'default': 'celsius',
     'description': 'The unit of temperature to return.',
     'enum': ['celsius', 'fahrenheit'],
     'title': 'Unit',
     'type': 'string'}},
   'required': ['city'],
   'title': 'GetWeatherArgs',
   'type': 'object'}},
 'callable': <function __main__.get_weather_func(city: str, unit: Literal['celsius', 'fahrenheit'] = 'celsius') -> str>}

## Custom Tools

### Regex tools
We can use a regex to define the type of structured input that our tool expects. In this example, our tool is a `whois` lookup that expects a well-formatted url.

Note that unlike `Tool.from_callable`, the callable passed to `Tool.from_regex` must take exactly one argument, which will be passed as a *string matching the given regex*.

In [4]:
import subprocess
from urllib.parse import urlparse

def whois_lookup(url: str) -> str:
    domain = urlparse(url).netloc
    result = subprocess.run(["whois", domain], capture_output=True, text=True, timeout=30)
    return result.stdout

whois_tool = Tool.from_regex(
    pattern=r"https?:\/\/[^\s]+",
    name="whois_lookup",
    description="A tool that performs a WHOIS lookup on a domain.",
    callable=whois_lookup
)
whois_tool.model_dump()

{'name': 'whois_lookup',
 'description': 'A tool that performs a WHOIS lookup on a domain.',
 'tool': {'type': 'custom',
  'format': {'type': 'grammar',
   'syntax': 'regex',
   'definition': 'https?:\\/\\/[^\\s]+'}},
 'callable': <function __main__.whois_lookup(url: str) -> str>}

### Guidance grammar tools
In addition to regex tools, we can define guidance grammars that constrain the model's output to match our tool's expected input format.

Below, we define a grammar that describes simple arithmetic expressions, and our tool will evaluate these expressions as python numbers.

Again, note that the callable passed to `Tool.from_grammar` must take exactly one argument, which will be passed as a *string matching the given grammar*.

In [5]:
from guidance import guidance, select, regex

@guidance(stateless=True)
def expression(lm):
    return lm + select([
        term(),
        expression() + "+" + term(),
        expression() + "-" + term(),
    ])

@guidance(stateless=True)
def term(lm):
    return lm + select([
        factor(),
        term() + "*" + factor(),
        term() + "/" + factor(),
    ])

@guidance(stateless=True)
def factor(lm):
    return lm + select([number(), "(" + expression() + ")"])

@guidance(stateless=True)
def number(lm):
    return lm + regex(r'\d+(\.\d+)?')

In [6]:
from sympy import sympify
from ast import literal_eval
from typing import Union

def evaluate_expression(expr: str) -> Union[int, float]:
    sympy_expr = sympify(expr)
    if not sympy_expr.is_number:
        raise ValueError(f"Invalid expression: {expr}")
    return literal_eval(str(sympy_expr))

calculator = Tool.from_grammar(
    grammar=expression(),
    name="calculator",
    description="A calculator that can evaluate mathematical expressions.",
    callable=evaluate_expression
)
calculator.model_dump()

{'name': 'calculator',
 'description': 'A calculator that can evaluate mathematical expressions.',
 'tool': {'type': 'custom',
  'format': {'type': 'grammar',
   'syntax': 'lark',
   'definition': '%llguidance {}\n\nstart: expression\n\nexpression: term\n     | expression "+" term\n     | expression "-" term\n\nterm: factor\n     | term "*" factor\n     | term "/" factor\n\nfactor: NUMBER\n     | "(" expression ")"\n\nNUMBER: /\\d+(\\.\\d+)?/\n'}},
 'callable': <function __main__.evaluate_expression(expr: str) -> Union[int, float]>}

### Lark tools
Under the hood, guidance grammars just compile down to a [lark-like syntax for describing context free grammars](https://github.com/guidance-ai/llguidance/blob/main/docs/syntax.md).

We can create a tool using this lark syntax directly. Note that the same caveats about arguments to the callable still apply.


In [7]:
lark = r"""
start: expression

expression: term
     | expression "+" term
     | expression "-" term

term: factor
     | term "*" factor
     | term "/" factor

factor: NUMBER
     | "(" expression ")"

NUMBER: /\d+(\.\d+)?/
"""

calculator_tool = Tool.from_lark(
    lark=lark,
    name="calculator",
    description="A calculator that can evaluate mathematical expressions.",
    callable=evaluate_expression
)
calculator_tool.model_dump()

{'name': 'calculator',
 'description': 'A calculator that can evaluate mathematical expressions.',
 'tool': {'type': 'custom',
  'format': {'type': 'grammar',
   'syntax': 'lark',
   'definition': '\nstart: expression\n\nexpression: term\n     | expression "+" term\n     | expression "-" term\n\nterm: factor\n     | term "*" factor\n     | term "/" factor\n\nfactor: NUMBER\n     | "(" expression ")"\n\nNUMBER: /\\d+(\\.\\d+)?/\n'}},
 'callable': <function __main__.evaluate_expression(expr: str) -> Union[int, float]>}

# Calling Tools

In [8]:
tools = [
    get_weather_tool,
    whois_tool,
    calculator_tool,
]

Most remote models have native support for tool-calling with function (JSON-specified) tools.

With OpenAI's release of GPT-5, they additionally support tools that are specified by regex or lark-style context free grammars. In these cases, model sampling is constrained using [LLGuidance](https://
github.com/guidance-ai/llguidance/blob/main/docs/syntax.md)

To allow our LM to use our tools, we simply pass them in a list as an argument to `gen`. If the LM wants to call one of our tools, `guidance` will handle the callback for us:

In [14]:
from guidance.models import OpenAI
from guidance import user, assistant, gen
lm = OpenAI("gpt-5-mini")

with user():
    lm += "What is the weather like in San Francisco?"
with assistant():
    lm += gen(tools=tools, tool_choice="required")
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

You may notice that the tool call is wrapped in `<function={name}>...</function>` tags and the result is wrapped in `<function_result>..</function_result>` tags. This is purely for visualization purposes, as OpenAI uses a structured representation under the hood which we (somewhat arbitrarily) represent as a string above.

In [15]:
lm = OpenAI("gpt-5-mini")

with user():
    lm += "If Johnny has 5 apples and gives 2 to Mary, how many apples does he have left?"
with assistant():
    lm += gen(tools=tools, tool_choice="required")
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [16]:
lm = OpenAI("gpt-5-mini")

with user():
    lm += "Who owns the openai website?"
with assistant():
    lm += gen(tools=tools, tool_choice="required")
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

## Local Models

We first define our tool-calling syntax in a system prompt.

If your model has been fine-tuned to follow a specific format, you may want to ensure that your definitions here are in-line with that format.

In [48]:
from json import dumps

system_prompt = f"""\
# Tool Instructions
You have access to the following functions:
{
    dumps(
        [
            {"name": tool.name, "description": tool.description, "arguments": tool.schema_dump()}
            for tool in tools
        ]
    )
}

If you choose to call a function, you must return the function name and arguments in the following format:
<function={{function_name}}>{{parameters}}</function>
where {{function_name}} is the name of the function you are calling, and {{parameters}} is a JSON object containing the parameters for the function call.

The return value of the function call will be provided to you in the following format:
<function_result>{{return_value}}</function_result>

After the return value is provided, you should summarize the information and provide a final response."""

Other imports and set-up

In [58]:
from guidance import models, system, user, assistant, gen
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="bartowski/Llama-3.2-3B-Instruct-GGUF",
    filename="Llama-3.2-3B-Instruct-Q6_K_L.gguf",
)

## The hard way

We now need to construct a grammar that enforces that the model behaves in-line with the system prompt that we wrote.

We first construct a grammar that ensures that any time the model produces the string `<function`, it is *forced* to
1. follow that with a valid function name
2. then follow the correct function's arguments

Here we will build this grammar directly using `llguidance`'s `lark`-like BNF syntax ([see documentation here](https://github.com/guidance-ai/llguidance/blob/main/docs/syntax.md#tool-calling)).

In [59]:
from IPython.display import display, Markdown

tool_grammar_raw = r"""start: TEXT | function_call
function_trg[lazy]: TEXT "<function="
function_end: "</function>"
TEXT: /(.|\n)*/"""

for tool in tools:
    tool_grammar_raw += """
function_call_{name}: function_{name}_begin function_{name}_args function_end
function_{name}_begin: function_trg "{name}>"
function_{name}_args: %json {schema}""".format(
    name=tool.name,
    schema=dumps(tool.schema_dump())
)

tool_grammar_raw += f"""
function_call[capture]: {" | ".join("function_call_" + tool.name for tool in tools)}
"""

display(Markdown(f"Tool grammar:\n```lark\n{tool_grammar_raw}\n```"))

Tool grammar:
```lark
start: TEXT | function_call
function_trg[lazy]: TEXT "<function="
function_end: "</function>"
TEXT: /(.|\n)*/
function_call_get_weather: function_get_weather_begin function_get_weather_args function_end
function_get_weather_begin: function_trg "get_weather>"
function_get_weather_args: %json {"additionalProperties": false, "properties": {"city": {"title": "City", "type": "string"}, "unit": {"enum": ["celsius", "fahrenheit"], "title": "Unit", "type": "string"}}, "required": ["city", "unit"], "title": "get_weather", "type": "object"}
function_call[capture]: function_call_get_weather

```

### Running the grammar

In [60]:
from guidance import lark

# Create the grammar from the raw string
tool_grammar = lark(tool_grammar_raw)
# Just double-checking that we wrote that correctly
tool_grammar._llguidance_validate()

lm = models.LlamaCpp(
    model_path,
    n_ctx=4096,
)

with system():
    lm += system_prompt
with user():
    lm += "What is the weather like in San Francisco?"
with assistant():
    lm += tool_grammar


llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

### Calling the function
Note that in the `llguidance` grammar, we captured the function call with the name "function_call"

In [61]:
print(lm["function_call"])

<function=get_weather>{"city": "San Francisco", "unit": "celsius"}</function>


In [62]:
import re
match = re.match(
    r"<function=(?P<name>[^>]+)>(?P<args>\{(.|\n)*\})</function>",
    lm["function_call"]
)
name = match.group("name")
args = match.group("args")
print(f"Function name: {name}")
print(f"Function arguments: {args}")

Function name: get_weather
Function arguments: {"city": "San Francisco", "unit": "celsius"}


In [63]:
matching_functions = [
    tool for tool in tools if tool.name == name
]
assert len(matching_functions) == 1, "Function name not found in tools"
function = matching_functions[0]
function_args = function.validate_args(args)
return_value = function.callable(**function_args)

print(f"Function return value: {return_value}")

Function return value: The current temperature in San Francisco is 19 degrees celsius and it is cloudy.


We have to add the function's return value to the model's context, as we promised.

Then we can finally let the model summarize the tool call.

In [64]:
with user():
    lm += f"<function_result>{return_value}</function_result>"
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

## The easy way

We can subclass `guidance.ToolCallHandler` in order to encapsulate some of the logic we had to manually write out above.

See the `CustomHandler` class defined below -- the methods you see defined here must be implemented by all subclasses.

In [ ]:
from guidance import ToolCallHandler, json as gen_json, gen
from guidance.types import Grammar
from typing import Any
import re
from json import dumps, loads


class CustomHandler(ToolCallHandler):
    def trigger(self):
        """
        A substring or token that signals when to start applying a tool.
        For example "<function" or "<|python_tag|>".
        """
        return "<function"

    def begin(self, tool_name: str) -> str:
        """
        The beginning of the tool call. Must start with trigger. May depend on the tool name.
        For example "<function=foo>", '<|python_tag|>{"name":"foo","parameters":', or just "<|python_tag|>".
        """
        return f"<function={tool_name}>"

    def body(self, tool: Tool) -> Grammar:
        """
        The body of the tool call. Should return a GrammarNode that matches the tool call arguments.
        For example, if begin contains the tool name, it may be given by
        ```python
            guidance.json(schema=tool.args.model_json_schema())
        ```
        or if begin does not contain the tool name, it may be given by
        ```python
        guidance.json(
            schema={
                "type": "object",
                "properties": {
                    "name": tool.name,
                    "parameters": tool.args.model_json_schema(),
                },
                "required": ["name", "parameters"],
                "additionalProperties": False,
            }
        )
        ```
        """
        return gen_json(schema=tool.schema.model_json_schema())

    def end(self) -> str:
        """
        The end of the tool call. Should return a string that ends the tool call.
        For example "</function><|eot_id|>\n" or "<|python_tag|><eom_id>".
        """
        return "</function>\n"

    def parse_tool_calls(self, text: str) -> tuple[str, dict[str, Any]]: 
        """
        Parse the tool calls from the text.
        Should return a list of tuples with tool name and args.
        """
        matches = re.finditer(
            r"<function=(?P<name>[^>]+)>(?P<args>\{(.|\n)*\})</function>",
            text
        )
        tool_calls = []
        for match in matches:
            tool_calls.append((match.group("name"), loads(match.group("args"))))
        return tool_calls

    def format_return_value(self, value: Any) -> str:
        """
        Format the return value of the tool call.
        Should return a string representation of the value.
        """
        return f"<function_result>{dumps(value)}</function_result>"

If we pass the tool call handler to our model init, we can then follow the same simple tool-calling pattern we used for remote models!

In [65]:
lm = models.LlamaCpp(
    model_path,
    n_ctx=4096,
    tool_call_handler_cls=CustomHandler
)

with system():
    lm += system_prompt
with user():
    lm += "What is the weather like in San Francisco?"
with assistant():
    lm += gen(tools=tools)
with assistant():
    lm += gen()

llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …